In [30]:
import pandas as pd
df = pd.read_csv("../../data/annonces-ventes-69.csv",delimiter=';',header=0,encoding="UTF-8",low_memory=False)
print (df.shape)
# display(df.head(20))
df = df [df.typedebien == "a"]
print (df.info())


(133369, 59)
<class 'pandas.core.frame.DataFrame'>
Index: 89190 entries, 6 to 133368
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   idannonce                89190 non-null  object 
 1   type_annonceur           89190 non-null  object 
 2   typedebien               89190 non-null  object 
 3   typedetransaction        89190 non-null  object 
 4   etage                    89190 non-null  int64  
 5   surface                  89190 non-null  int64  
 6   surface_terrain          3219 non-null   float64
 7   nb_pieces                89190 non-null  int64  
 8   prix_bien                89190 non-null  int64  
 9   prix_maison              0 non-null      float64
 10  prix_terrain             0 non-null      float64
 11  mensualiteFinance        89190 non-null  int64  
 12  balcon                   89190 non-null  int64  
 13  eau                      89190 non-null  int64  
 14  bain         

In [ ]:
# drop empy columns
print (df.shape)
# threshold = 90%
threshold = int(df.shape[0]*0.9)
empty_cols = []
for name,_ in df.items() :
    if df[name].isna().sum() > threshold :
        empty_cols.append(name) 
print (f"empty cols {empty_cols}")
# remove empty cols
df = df.drop(columns=empty_cols)
print(df.isna().sum())
# remove unnecessary columns
columns = ["typedebien","typedetransaction","mensualiteFinance","nb_terraces","REG","DEP"]
df = df.drop(columns=columns)
# check for columns content
for column in df.columns:
    print(f"{column}: {df[column].unique()}")
columns = ["type_annonceur","typedebien_lite"]
df = df.drop(columns=columns)

(89190, 59)
empty cols ['surface_terrain', 'prix_maison', 'prix_terrain', 'parking', 'videophone', 'surface_balcon']
idannonce                      0
type_annonceur                 0
typedebien                     0
typedetransaction              0
etage                          0
surface                        0
nb_pieces                      0
prix_bien                      0
mensualiteFinance              0
balcon                         0
eau                            0
bain                           0
dpeL                           0
dpeC                       22455
mapCoordonneesLatitude         0
mapCoordonneesLongitude        0
annonce_exclusive              0
nb_etages                  28163
places_parking             60456
cave                       50236
exposition                     0
ges_class                  15799
annee_construction         55046
nb_toilettes               40830
nb_terraces                73392
porte_digicode             68759
ascenseur                

In [ ]:
#keep rows with no Nans
#keep rows with no Nans
subset = ["chauffage_mode","chauffage_systeme","chauffage_energie","annee_construction","places_parking","cave","charges_copro","nb_logements_copro","ascenseur"]
df= df.dropna(subset=subset)
print(df.shape)
df.isna().sum()
df.info()

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#         display(df.head(100))


In [47]:
# clean column nb_logements_copro
df["nb_logements_copro"] = df["nb_logements_copro"].fillna(df["nb_logements_copro"].mean()).astype("int")
df["nb_logements_copro"].unique()

df["annee_construction"] = df["annee_construction"].astype("int")
df["annee_construction"].unique()

df["places_parking"] = df["places_parking"].astype("int")
df["places_parking"].unique()

df["dpeL"] = df["dpeL"].replace(to_replace=["D - 231 kWh/m².year","D-218 kWh/m².year","Blank"],value=["D","D","D"])
df["dpeL"].unique()


array(['D', 'C', 'E', 'VI', 'NS', 'B', 'F', 'A', '0', 'G'], dtype=object)